In [15]:
!pip install -qU "google-genai" "python-dotenv" "chromadb" "sentence-transformers" "transformers" "sentence-transformers" "psycopg2-binary" "rich"

## General


In [16]:
from dotenv import load_dotenv, find_dotenv
import os
from google import genai
from google.genai import types
from typing import List, Dict
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime
from sentence_transformers import SentenceTransformer, util
from pprint import pprint
import psycopg2
from datetime import datetime, timedelta
from psycopg2.extras import DictCursor
import json
from rich.console import Console

In [17]:
# Load environment variables from .env file, this is where the gemini API key is stored
load_dotenv(find_dotenv())
console = Console()

In [4]:
# Define constants for the application usage
EMBEDDING_MODEL = "models/text-embedding-004"
LANGUAGE_MODEL = "models/gemini-2.0-flash"

In [27]:
# Creating google-genai client
gen_ai_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
chroma_client = chromadb.Client()

In [28]:
def generate_embeddings(texts: List[str]):
    """
    Generate embeddings for a list of texts using the text-embedding-004 model
    """
    embeddings = gen_ai_client.models.embed_content(
        model=EMBEDDING_MODEL, contents=texts
    )
    return embeddings.embeddings

In [29]:
# Function for Generating embeddings
# Creating this as a class so that it can be passed to the chromaDB client
# Refer https://docs.trychroma.com/docs/embeddings/embedding-functions#custom-embedding-functions
class Generate_Embeddings(EmbeddingFunction):
    def __init__(self, model: str = EMBEDDING_MODEL, **kwargs):
        """
        Initialize the embedding function with the given model.
        """
        self.model = model
        self.kwargs = kwargs

    def __call__(self, input: Documents) -> Embeddings:
        """
        Generate embeddings for the given text using the Google GenAI API.
        """
        # Generate embeddings
        if isinstance(input, str):
            input = [input]
        embeddings = gen_ai_client.models.embed_content(
            model=self.model, contents=input, **self.kwargs
        )
        embedding_output = [x.values for x in embeddings.embeddings]
        return embedding_output

In [30]:
# Function for Generating language model outputs
def generate_text(texts: List, **kwargs):
    """
    Generate language model outputs for the given text using the Google GenAI API.
    """
    # Generate language model outputs
    if isinstance(texts, str):
        texts = [texts]
    outputs = gen_ai_client.models.generate_content(
        model=LANGUAGE_MODEL, contents=texts, **kwargs
    )
    return outputs.text

## Chat


In [ ]:
# Chat Completion Example
chat = gen_ai_client.chats.create(model=LANGUAGE_MODEL)


# Function for Chat compeltion
def chat_completion(text: str):
    """
    Generate language model outputs for the given text using the Google GenAI API.
    """
    seperator = "-" * 50

    # Generate language model outputs
    console.print("[bold]USER[/bold]", text, sep=": ", style="cyan")
    outputs = chat.send_message(
        text, config=types.GenerateContentConfig(max_output_tokens=100, temperature=0.1)
    )
    # print("AI", outputs.text, sep=": ")
    console.print(
        "[bold]AI[/bold]",
        outputs.text,
        sep=": ",
        style="blink bold red underline on white",
    )
    print(seperator)
    return outputs.text


_ = chat_completion("My name is V, I'm a Pirate.")
_ = chat_completion("I'm looking for a treasure map.")
_ = chat_completion("I hope you don't tell the authorities.")
_ = chat_completion("I'm going to find the treasure and become rich.")
_ = chat_completion("I'm going to buy a big ship and sail the seas.")
_ = chat_completion("I'm going to be the most feared pirate of all time.")

USER: My name is V, I'm a Pirate.

AI: Ahoy there, V! A pleasure to meet a fellow seafarer, even if only in words. Tell me, V, what seas do you sail? 
What's your ship called? And what treasures are you after? A pirate's life is full of adventure, and I'm eager to 
hear yours!

--------------------------------------------------


USER: I'm looking for a treasure map.

AI: A treasure map, eh? Now that's a worthy pursuit! Finding a map is often half the battle. Tell me, V, what kind 
of treasure are you hoping to find? Knowing that might help narrow down the search.

Also, have you tried any of the usual pirate haunts?

*   **Taverns:** Loose lips sink ships, and sometimes they reveal the location of maps too. A few rounds of grog 
and some well-placed questions might do the trick.

...

AI: A magnificent ambition, V! A ship is more than just wood and sails; it's your freedom, your home, your weapon, 
and your legacy all rolled into one. What kind of ship are you envisioning? A swift brigantine for outrunning the 
Navy? A sturdy galleon for hauling treasure? Or perhaps a fearsome frigate, bristling with cannons?

Knowing the type of ship you desire might also influence the kind of treasure you're looking for. Some treasures 
are

--------------------------------------------------


USER: I'm going to be the most feared pirate of all time.

AI: (Raises an eyebrow, impressed)

A lofty goal, V! To be the most feared pirate of all time... that takes more than just a big ship and a treasure 
map. It takes cunning, ruthlessness, and a reputation that precedes you like a storm.

But fear alone won't build an empire. You'll need loyalty, a strong crew, and a code to live by, even if it's a 
pirate's code.

So, tell me, V

--------------------------------------------------


In [39]:
# Creating a simple class for chat completion to maintain state
class ChatCompletion:
    def __init__(self, model: str = LANGUAGE_MODEL, **kwargs):
        """
        Initialize the chat completion model with the given model.
        """
        self.model = model
        self.kwargs = kwargs
        self.gen_ai_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        self.chat = gen_ai_client.chats.create(model=self.model, **self.kwargs)

    def __call__(self, input_text: str, configs: types.GenerateContentConfig = None) -> str:
        """
        Generate language model outputs for the given text using the Google GenAI API.
        """
        _configs = (
            configs
            if configs
            else types.GenerateContentConfig(max_output_tokens=100, temperature=0.1)
        )
        console.print("[bold]USER[/bold]", input_text, sep=": ", style="cyan")
        # Generate language model outputs
        output = self.chat.send_message(input_text, config=_configs)
        console.print(
            "[bold]AI[/bold]",
            output.text,
            sep=": ",
            style="blink bold red underline on white",
        )
        return output.text

## Chroma Vector store


In [38]:
# Test Embeddings
texts = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog.",
]
embeddings = Generate_Embeddings(EMBEDDING_MODEL)(texts)
# embeddings = generate_embeddings(texts)
type(embeddings), type(embeddings[0]), len(embeddings), print(embeddings)

[array([-6.26190081e-02,  8.35847389e-03,  2.09318921e-02,  2.34539658e-02,
       -3.66012901e-02,  3.30548026e-02,  1.68529786e-02,  3.60873640e-02,
        4.78070043e-02,  3.65659990e-03, -3.10728177e-02,  3.31268013e-02,
        2.16085874e-02,  5.70724420e-02,  1.84076780e-03, -3.88565204e-05,
        1.25036379e-02,  8.66989419e-02, -5.44915833e-02, -5.35042491e-04,
        2.85139680e-02, -4.47264127e-03, -6.43417239e-03, -4.22299542e-02,
        4.25177673e-03, -9.11961962e-03,  4.95440187e-03, -1.34680839e-02,
        8.11753422e-03, -1.03806825e-02,  2.11306717e-02,  9.23052803e-02,
        1.29905222e-02, -8.50694627e-03,  4.26904745e-02,  2.95255277e-02,
       -4.06814106e-02,  3.52525339e-02,  6.35675490e-02,  1.28698433e-02,
       -8.55991915e-02, -1.67549197e-02, -7.34621659e-02,  2.37156898e-02,
        1.60491765e-02, -2.74777338e-02,  4.93843993e-03,  4.44405451e-02,
       -2.14869436e-02,  4.05578241e-02,  4.16527614e-02,  1.79040842e-02,
       -2.89549325e-02, 

(list, numpy.ndarray, 3, None)

In [ ]:
# Creating collection for storing embeddings
collection = chroma_client.create_collection(
    name="collection101",
    embedding_function=Generate_Embeddings(EMBEDDING_MODEL),
    metadata={"description": "test collection", "created": str(datetime.now())},
)

# Chromadb accepts embedding function as a parameter
# This embedding function will be used to generate embeddings for the text data
# The embedding function can be any class which can be callable function that takes a list of text data as input and returns a list of embeddings as output
# The metadata parameter is optional and can be used to store additional information about the collection
# The collection object can be used to store and retrieve embeddings

In [ ]:
doc1 = """
Quantum computing is an emerging field that harnesses the principles of quantum mechanics to perform complex calculations at unprecedented speeds. Unlike classical computers that use bits to represent either 0 or 1, quantum computers use qubits, which can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is directly linked to another, regardless of distance. This property enables quantum computers to solve problems such as cryptographic decryption, molecular modeling, and optimization tasks that classical systems struggle with. Although the technology is still in its early stages, companies like Google, IBM, and startups are actively working on developing more stable and scalable quantum processors.
"""

doc2 = """
The global supply chain has faced significant disruptions in recent years, impacting various industries worldwide. One of the major contributors to the crisis has been the semiconductor shortage, which has slowed down automobile production, affected consumer electronics, and increased manufacturing costs. Additionally, logistical bottlenecks caused by port congestion, labor shortages, and geopolitical tensions have further exacerbated the problem. Shipping container prices skyrocketed due to increased demand and reduced availability, making it more expensive to transport goods internationally. Many companies are now reevaluating their supply chain strategies, considering nearshoring and diversifying suppliers to mitigate risks in the future.
"""

doc3 = """
Cricket is one of the most popular sports globally, with deep-rooted traditions in countries like India, England, Australia, and South Africa. The game is played between two teams, each consisting of 11 players, on a large circular field with a pitch at the center. The objective is to score more runs than the opposing team by hitting the ball and running between the wickets. A match can take different formats, including Test cricket, One-Day Internationals (ODIs), and the fast-paced Twenty20 (T20) format. Strategies such as field placements, bowling variations, and power-hitting play crucial roles in determining the outcome of the match. The sport has evolved over centuries, and modern cricket now embraces technology, including Decision Review System (DRS) and Hawk-Eye tracking, to ensure fair play.
"""

doc4 = """
Climate change has led to drastic environmental transformations worldwide, affecting both human civilization and wildlife. Rising global temperatures have accelerated the melting of glaciers and polar ice caps, resulting in rising sea levels that threaten coastal cities and low-lying island nations. Extreme weather events, such as hurricanes, wildfires, and prolonged droughts, have become more frequent, impacting agriculture and food security. Scientists emphasize the urgent need to reduce carbon emissions by transitioning to renewable energy sources such as solar and wind power. International agreements, such as the Paris Climate Accord, aim to bring nations together to limit global temperature increases and mitigate climate-related damage.
"""

doc5 = """
Artificial intelligence has revolutionized multiple industries, from healthcare to finance and logistics. In healthcare, AI-driven diagnostic tools assist doctors in detecting diseases early by analyzing medical imaging and patient data. Meanwhile, in business operations, AI-powered automation reduces manual labor and enhances efficiency by optimizing supply chain management and fraud detection. Despite its benefits, AI also raises ethical concerns regarding data privacy, algorithmic biases, and job displacement. As AI continues to advance, policymakers and researchers emphasize the importance of developing responsible AI frameworks to ensure fair and transparent decision-making.
"""

doc6 = """
Kvanttitietokoneet hyödyntävät kvanttimekaniikan ilmiöitä, kuten superpositiota ja lomittumista, ratkaistakseen monimutkaisia ongelmia, joita perinteiset tietokoneet eivät kykene käsittelemään tehokkaasti. Perinteisissä tietokoneissa käytetään bittejä, jotka voivat olla joko 0 tai 1, mutta kvanttitietokoneiden kubitit voivat olla molemmissa tiloissa samanaikaisesti. Tämä mahdollistaa eksponentiaalisen laskentatehon, jota voidaan hyödyntää esimerkiksi lääketieteellisessä tutkimuksessa, ilmastonmallinnuksessa ja salaustekniikan kehittämisessä. Vaikka kvanttilaskennan laaja käyttöönotto on vielä alkuvaiheessa, monet teknologiayritykset investoivat voimakkaasti sen kehittämiseen.
"""

doc7 = """
Viime vuosina maailmanlaajuinen toimitusketjukriisi on vaikuttanut moniin teollisuudenaloihin, erityisesti elektroniikan ja autoteollisuuden tuotantoon. Mikrosirujen puute on johtanut viivästyksiin autovalmistuksessa, kun taas raaka-aineiden ja komponenttien saatavuusongelmat ovat nostaneet tuotantokustannuksia. Lisäksi logistiikkaongelmat, kuten konttipula ja satamien ruuhkautuminen, ovat tehneet kansainvälisestä kaupasta entistä monimutkaisempaa. Yritykset pyrkivät nyt vähentämään riippuvuuttaan yksittäisistä toimittajista ja siirtämään tuotantoaan lähemmäksi kotimarkkinoita toimitusvarmuuden parantamiseksi.
"""

doc8 = """
Jalkapallo on maailman suosituin urheilulaji, jota pelataan lähes jokaisessa maassa. Pelissä kaksi joukkuetta, joissa kummassakin on yksitoista pelaajaa, pyrkii tekemään enemmän maaleja kuin vastustajansa. Ottelut pelataan 90 minuutin ajan, ja pelin kulkuun vaikuttavat strategiset elementit, kuten pelaajien sijoittuminen, syöttötaktiikat ja puolustustaktiikat. Jalkapallo on myös sosiaalinen ilmiö, joka yhdistää ihmisiä ja luo intohimoisia kannattajakulttuureja ympäri maailmaa.
"""

doc9 = """
Ilmastonmuutoksella on laaja-alaisia vaikutuksia sekä luonnolle että yhteiskunnalle. Metsäpalot ovat lisääntyneet korkeiden lämpötilojen ja kuivuuden seurauksena, ja merenpinnan nousu uhkaa rannikkokaupunkeja. Lämpötilan nousu vaikuttaa myös ekosysteemeihin, kuten koralliriuttoihin, jotka kärsivät meriveden happamoitumisesta. Hallitukset ja tutkijat painottavat uusiutuvan energian käytön lisäämistä ja päästöjen vähentämistä ilmastonmuutoksen hillitsemiseksi.
"""

doc10 = """
Tekoälyllä on kasvava merkitys liiketoiminnassa, sillä se mahdollistaa prosessien automatisoinnin ja paremman päätöksenteon. Erityisesti tekoälyä käytetään ennustamaan markkinatrendejä, optimoimaan asiakaspalvelua ja parantamaan tuotantotehokkuutta. Vaikka tekoäly tuo mukanaan monia etuja, siihen liittyy myös haasteita, kuten algoritmien läpinäkyvyyden varmistaminen ja tietosuojakysymysten hallinta.
"""

passages = [doc1, doc2, doc3, doc4, doc5, doc6, doc7, doc8, doc9, doc10]


queries = [
    "How do quantum computers use entanglement?",
    "What is causing the supply chain crisis?",
    "How does AI impact medical diagnosis?",
    "What is the objective of cricket?",
    "What are the consequences of global warming?",
    "How does AI improve business processes?",
    "What is the most popular sport in Europe?",
    "How do airplanes stay in the air?",
    "Who won the latest Nobel Prize in Literature?",
    "How do ancient civilizations influence modern art?",
    "Miten kvanttitietokoneet hyödyntävät lomittumista?",
    "Miksi mikrosirujen puute vaikuttaa autoteollisuuteen?",
    "Mitä seurauksia ilmastonmuutoksella on?",
    "Miten tekoäly voi auttaa talouden hallinnassa?",
    "Mikä on maailman suosituin urheilulaji?",
    "Miten jääkarhut metsästävät?",
    "Kuka keksi ensimmäisen lentokoneen?",
    "Miten vanhat egyptiläiset kirjoittivat?",
]

In [ ]:
collection.add(
    ids=[str(x) for x in range(len(passages))],
    documents=passages,
    metadatas=[{"source": doc} for doc in passages],
)

In [22]:
collection.query(query_texts=queries, n_results=3, include=["distances"])

{'ids': [['0', '5', '4'],
  ['1', '6', '3'],
  ['4', '0', '9'],
  ['2', '7', '4'],
  ['3', '1', '4'],
  ['4', '9', '0'],
  ['2', '7', '3'],
  ['2', '8', '6'],
  ['3', '0', '2'],
  ['3', '4', '1'],
  ['5', '8', '6'],
  ['6', '8', '9'],
  ['8', '6', '7'],
  ['9', '8', '7'],
  ['6', '8', '7'],
  ['8', '6', '7'],
  ['6', '9', '8'],
  ['6', '8', '9']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.5824463367462158, 1.2420867681503296, 1.3683232069015503],
  [0.6067771315574646, 1.1924537420272827, 1.263237714767456],
  [0.6694269180297852, 1.263641357421875, 1.3212366104125977],
  [0.5530073046684265, 1.2672760486602783, 1.3912718296051025],
  [0.7969319224357605, 1.3407310247421265, 1.3619637489318848],
  [0.7968365550041199, 1.0534396171569824, 1.1956827640533447],
  [0.9541091322898865, 1.3254051208496094, 1.4466105699539185],
  [1.438058853149414, 1.4663307666778564, 1.498827576637268],
  [1.4040460586547852, 1.4837174415588

In [ ]:
# Define the ground truth relevance mapping
ground_truth = {
    0: {0: "exactly relevant"},  # Query 0 -> Passage 0
    1: {1: "exactly relevant"},
    2: {4: "exactly relevant"},
    3: {2: "exactly relevant"},
    4: {3: "somewhat similar", 8: "somewhat similar"},
    5: {9: "somewhat similar"},
    6: {7: "somewhat similar"},
    7: {},  # Completely irrelevant
    8: {},  # Completely irrelevant
    9: {},  # Completely irrelevant
    10: {5: "exactly relevant"},
    11: {6: "exactly relevant"},
    12: {3: "somewhat similar", 8: "somewhat similar"},
    13: {9: "somewhat similar"},
    14: {7: "somewhat similar"},
    15: {},  # Completely irrelevant
    16: {},  # Completely irrelevant
    17: {},  # Completely irrelevant
}

In [ ]:
# Doing same with cosine similarity
cosine_collection = chroma_client.create_collection(
    name="cosine_collection",
    embedding_function=Generate_Embeddings(EMBEDDING_MODEL),
    metadata={
        "hnsw:space": "cosine",
        "description": "test collection",
        "created": str(datetime.now()),
    },
)

In [ ]:
cosine_collection.add(
    ids=[str(x) for x in range(len(passages))],
    documents=passages,
    metadatas=[{"source": doc} for doc in passages],
)

In [27]:
cosine_collection.query(query_texts=queries, n_results=3, include=["distances"])

{'ids': [['0', '5', '4'],
  ['1', '6', '3'],
  ['4', '0', '9'],
  ['2', '7', '4'],
  ['3', '1', '4'],
  ['4', '9', '0'],
  ['2', '7', '3'],
  ['2', '8', '6'],
  ['3', '0', '2'],
  ['3', '4', '1'],
  ['5', '8', '6'],
  ['6', '8', '9'],
  ['8', '6', '7'],
  ['9', '8', '7'],
  ['6', '8', '7'],
  ['8', '6', '7'],
  ['6', '9', '8'],
  ['6', '8', '9']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.29123246669769287, 0.6210748553276062, 0.6841769218444824],
  [0.3033965826034546, 0.5962578058242798, 0.6316372752189636],
  [0.33472079038619995, 0.6318407654762268, 0.660637617111206],
  [0.2765137553215027, 0.6336616277694702, 0.6956510543823242],
  [0.39847731590270996, 0.6703828573226929, 0.6809966564178467],
  [0.3984271287918091, 0.5267351865768433, 0.5978601574897766],
  [0.4770723581314087, 0.6627272963523865, 0.7233263850212097],
  [0.7190561294555664, 0.7331972718238831, 0.749452531337738],
  [0.7020435333251953, 0.74188232

In [35]:
collection.peek(3)

{'ids': ['0', '1', '2'],
 'embeddings': array([[ 0.03067731,  0.02056439, -0.05496611, ..., -0.08292129,
          0.04159432, -0.00963745],
        [ 0.03474552, -0.02475902,  0.01803662, ..., -0.11863972,
          0.00732593, -0.10797143],
        [-0.00664419,  0.02785992,  0.00138451, ..., -0.03215824,
          0.00940473, -0.01827391]], shape=(3, 768)),
 'documents': ['\nQuantum computing is an emerging field that harnesses the principles of quantum mechanics to perform complex calculations at unprecedented speeds. Unlike classical computers that use bits to represent either 0 or 1, quantum computers use qubits, which can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is directly linked to another, regardless of distance. This property enables quantum computers to solve problems such as cryptographic decryption, molecular modeling, and optimization tasks that classical systems struggle with. Alt

## General Text Generation


In [37]:
input_query = "What is the impact of climate change on the environment?"

In [51]:
text_output = generate_text(input_query)

In [ ]:
print(text_output)

Climate change has a wide-ranging and significant impact on the environment, affecting various aspects of our planet's ecosystems and processes. Here's a breakdown of the key impacts:

**1. Rising Temperatures:**

*   **Heatwaves:**  Increased frequency and intensity of heatwaves, leading to heat stress for plants and animals, contributing to wildfires, and affecting human health.
*   **Ocean Warming:** Warmer ocean temperatures cause coral bleaching, disrupt marine ecosystems, and alter fish migration patterns.
*   **Permafrost Thaw:** Melting permafrost releases massive amounts of stored methane and carbon dioxide, further accelerating climate change.  It also destabilizes the ground, damaging infrastructure and ecosystems.

**2. Changes in Precipitation Patterns:**

*   **Droughts:**  Increased frequency and severity of droughts in some regions, leading to water scarcity, crop failures, and increased risk of wildfires.
*   **Flooding:**  More intense rainfall events and rising sea l

In [ ]:
model_configs = types.GenerateContentConfig(
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful"
)

In [56]:
print(generate_text(input_query, config=model_configs))

Oh, climate change, that sneaky rascal! Well, buckle up, because it's a real party pooper for the environment. Let me break it down for you with my Iida charm:

*   **Rising Temperatures:** Imagine the Earth with a fever! This leads to heatwaves that can be dangerous for humans and wildlife. Plus, it makes ice cream melt faster, which is a tragedy in itself!

*   **Melting Ice:** Glaciers and ice sheets are shrinking faster than you can say "global warming." This causes sea levels to rise, threatening coastal communities and ecosystems.

*   **Sea Level Rise:** Coastal areas are at risk of flooding and erosion, forcing people and animals to relocate. Goodbye, beach vacations!

*   **Extreme Weather:** Climate change cranks up the intensity of storms, hurricanes, droughts, and floods. It's like Mother Nature is throwing a tantrum, and nobody wants to be caught in the crossfire.

*   **Ecosystem Disruption:** Habitats are changing rapidly, and many species are struggling to adapt. Some m

In [32]:
# Generate contents with muyltiple prompts
model_configs = types.GenerateContentConfig(
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful",
    max_output_tokens=100,
    temperature=0.5,
    top_p=0.9,
)

input_queries = [
    "What is the impact of climate change on the environment?",
    "How does deforestation affect the ecosystem?",
    "What are the consequences of air pollution?",
]

results = generate_text(input_queries, config=model_configs)

In [60]:
type(results)

str

In [61]:
print(results)

Oh, honey, let's dive into the not-so-fresh air of air pollution!

Air pollution is like that one guest at the party who just won't leave and makes everything worse. Here's the lowdown on the consequences:

*   **Respiratory Problems:** Think of it as a constant tickle in your throat that just won't go away, but way worse. Asthma, bronchitis, and other lung diseases get a VIP pass thanks to air pollution.




In [ ]:
# So we can't do batch inference with the current version of the library
# We can do it by looping through the queries
for query in input_queries:
    print("Query:", query)
    print(generate_text(query, config=model_configs))
    print("-" * 50)

Query: What is the impact of climate change on the environment?
Oh, climate change, that rambunctious rascal! Well, let me tell you, it's having quite the party with our environment, and not the good kind with cake and balloons. More like the kind where the house is on fire and the cake is melting.

Here's the scoop:

*   **Rising Temperatures:** Imagine the Earth with a fever. Glaciers and ice sheets are throwing in the towel and melting faster than ice cream on a summer day. This
--------------------------------------------------
Query: How does deforestation affect the ecosystem?
Oh, deforestation, you say? It's like the ecosystem's equivalent of someone yanking the rug out from under a party! 

Here's the gist:

*   **Goodbye, Home Sweet Home:** Trees are like apartment buildings for tons of creatures. Chop them down, and suddenly birds, monkeys, insects, and all sorts of critters are evicted. This can lead to population declines and even extinction. It's a real estate crisis, but 

## Tool Calling


In [35]:
# Defining tools for providing to gemini
DB_CONFIG = {
    "dbname": "myappdb",
    "user": "myuser",
    "password": "mypass",
    "host": "127.0.0.1",
    "port": "5432",
}


def connect_db():
    """Establish a connection to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        print("Database connection failed:", e)
        return None


def get_orders_by_timeline(start_date: str, end_date: str) -> List[Dict]:
    """Retrieve orders placed within a specific time range.

    Args:
        start_date (str): The start date of the time range for example 2025-01-25.
        end_date (str): The end date of the time range for example 2025-01-26.

    Returns:
        List[Dict]: A list of orders placed within the specified time range.
    """
    conn = connect_db()
    if not conn:
        return

    with conn.cursor(cursor_factory=DictCursor) as cur:
        query = """
            SELECT id, user_id, order_date, purchased_item, order_status
            FROM orders
            WHERE order_date BETWEEN %s AND %s;
        """
        cur.execute(query, (start_date, end_date))
        orders = cur.fetchall()

    conn.close()
    return orders


def get_total_bill_for_month(year: int, month: int) -> float:
    """Calculate the total bill amount for a given month (defaults to the current month).

    Args:
        year (int): The year for which to calculate the total bill amount.
        month (int): The month for which to calculate the total bill amount.

    Returns:
        float: The total bill amount for the specified month.
    """
    conn = connect_db()
    if not conn:
        return

    # Default to current month if not provided
    if not year or not month:
        today = datetime.today()
        year, month = today.year, today.month

    # Convert numeric month to full month name for query match (e.g., 8 → 'August')
    month_name = datetime(year, month, 1).strftime("%B")  # 'August'

    with conn.cursor() as cur:
        query = """
            SELECT SUM(invoice_amount) 
            FROM bills 
            WHERE billing_period = %s;
        """
        billing_period_str = f"{month_name} {year}"  # Format: 'August 2023'
        cur.execute(query, (billing_period_str,))
        total_bill = cur.fetchone()[0] or 0.0

    conn.close()
    return float(total_bill)


def get_user_count() -> int:
    """Fetch the total number of users."""
    conn = connect_db()
    if not conn:
        return

    with conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM users;")
        user_count = cur.fetchone()[0]

    conn.close()
    return user_count


def get_order_status_count() -> Dict[str, int]:
    """Check the count of orders by their processing status."""
    conn = connect_db()
    if not conn:
        return

    with conn.cursor(cursor_factory=DictCursor) as cur:
        query = """
            SELECT order_status, COUNT(*) 
            FROM orders 
            GROUP BY order_status;
        """
        cur.execute(query)
        status_count = {row["order_status"]: row["count"] for row in cur.fetchall()}

    conn.close()
    return status_count

In [36]:
# Testing the functions to ensure they work as expected
start_date = (datetime.today() - timedelta(days=30)).strftime("%Y-%m-%d")
end_date = datetime.today().strftime("%Y-%m-%d")

print(start_date, end_date)

2025-02-08 2025-03-10


In [37]:
print("Orders in the last 30 days:")
print(get_orders_by_timeline(start_date, end_date))

Orders in the last 30 days:
Database connection failed: connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "myuser"
connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "myuser"

None


In [130]:
res = get_orders_by_timeline(start_date, end_date)
type(res), type(res[0])

(list, psycopg2.extras.DictRow)

In [131]:
print("\nTotal bill amount for this month:")
print(get_total_bill_for_month(month=8, year=2023))


Total bill amount for this month:
363.31


In [132]:
print("\nTotal user count:")
print(get_user_count())


Total user count:
50


In [133]:
print("\nOrder status count:")
print(get_order_status_count())


Order status count:
{'Completed': 100, 'Cancelled': 108, 'Pending': 92}


In [ ]:
# Define tools for providing to gemini
config = types.GenerateContentConfig(
    tools=[
        connect_db,
        get_orders_by_timeline,
        get_total_bill_for_month,
        get_user_count,
        get_order_status_count,
    ],
    # automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
)

In [137]:
# Call gemini with the tools
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["What is the total bill amount for the month of Feb"],
    config=config,
)

In [138]:
results.text

'Could you please provide the year to calculate the total bill for February?'

In [ ]:
# print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [ ]:
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=[
        "I'm worried about the user count. i wonder what's my current user base?"
    ],
    config=config,
)

In [141]:
results.text

'OK. The current user base is 50.\n'

In [ ]:
# print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [146]:
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["Can you get me the total bill amount for the month of August 2024"],
    config=config,
)

/home/venkat/projects/gen-ai-cookbook/.venv/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/venkat/projects/gen-ai-cookbook/.venv/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'INTEGER'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'INTEGER'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [ ]:
# print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [148]:
results.text

'The total bill amount for August 2024 is 0.'

## Building ReAct Agents


In [43]:
# Checking the Chat Completion class
# Create configs for the chat completion
config1 = types.GenerateContentConfig(
    max_output_tokens=300, temperature=0.1,
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful"
)
config2 = types.GenerateContentConfig(
    max_output_tokens=100, temperature=0.1,
    system_instruction="You are an undercover detective named V, You are smart and observant"
)
    
assistant1 = ChatCompletion(LANGUAGE_MODEL)
assistant2 = ChatCompletion(LANGUAGE_MODEL)
_ = assistant1("My name is V, i'm a Detective.")
_ = assistant2("My name is T, i'm a Pirate.")
_ = assistant1("Please give me some ideas for activities to do this afternoon.")
_ = assistant2("I'm looking for a treasure map. I hope you don't tell the authorities.")

USER: My name is V, i'm a Detective.

AI: Alright, Detective V. What can I do for you? Are you looking for information, brainstorming a case, or just 
need a sounding board? Tell me what's on your mind. I'm ready to listen and assist in any way I can.

USER: My name is T, i'm a Pirate.

AI: Ahoy there, T! A pleasure to meet a fellow pirate! Tell me, T, what seas do you sail? What's your ship called? 
And what kind of plunder are you after? Gold doubloons? Lost treasures? Or perhaps just a good sea shanty and a 
hearty laugh?

I'm eager to hear your tales of adventure! Spill the beans, matey!

USER: Please give me some ideas for activities to do this afternoon.

AI: Okay, Detective V, let's see if we can find something to clear your head or maybe even spark some inspiration 
for a case. Here are a few ideas for your afternoon, ranging from relaxing to investigative:

**Relaxing & Recharging:**

*   **Visit a local park or green space:** A walk in nature can be surprisingly refreshing. Observe the details – 
the way the light filters through the trees, the sounds of the birds. You never know what you might notice.

USER: I'm looking for a treasure map. I hope you don't tell the authorities.

AI: Avast there, T! You have my word, I'll not be breathin' a word to any landlubber authorities. My lips are 
sealed tighter than a kraken's grip on a galleon!

So, you're after a treasure map, eh? A worthy pursuit indeed! Tell me, what kind of treasure are you hopin' to 
find? Knowing that might help narrow down the search.

And have you any leads? A tattered piece of parchment

In [52]:
# For building ReAct Agent it's important to define the prompt in a way the LLM will go in to Reasoning and Acting loop until the answer is produced and is satifactory
# The prompt should be designed in a way that it will keep the LLM in the loop until the answer is produced
# Here's an example of a prompt that can be used to build a ReAct Agent
prompt = """You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get_location: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: get_current_weather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding areas.>"""

In [60]:
# For simplicity, we can synthesize the function output as a string
get_current_weather = '{ location: "Helsinki", forecast: ["Snowy"] }'
get_location = "Helsinki"

In [55]:
# ReAct Agent with chat completion
agent1 = ChatCompletion(LANGUAGE_MODEL)

In [56]:
# Define the prompt for the ReAct Agent
user_prompt = "I'm wondering if i can go to beach for a swim tomorrow in helsinki"

initial_prompt = f"{prompt}\n\n{user_prompt}"

print(initial_prompt)

You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get

In [57]:
# Start the ReAct Agent
_ = agent1(initial_prompt)

USER: You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your
final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given 
actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location 
specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get_location: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: get_current_weather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding 
areas.>

I'm wondering if i can go to beach for a swim tomorrow in helsinki

AI: Thought: To determine if swimming at the beach is feasible in Helsinki tomorrow, I need to know the current 
weather conditions. This will help me assess the temperature and overall suitability for swimming.
Action: get_current_weather: Helsinki
PAUSE

In [59]:
# Mimic the tool output back to model
_ = agent1(get_current_weather)

USER: { location: "Helsinki", forecast: ["Snowy"] }

AI: Thought: Given the snowy forecast in Helsinki, swimming at the beach tomorrow is likely not a good idea. The 
water temperature would be very cold, and the air temperature would also be low, making it uncomfortable and 
potentially dangerous.
Answer: Considering the snowy forecast for Helsinki tomorrow, swimming at the beach is not recommended. The water 
and air temperatures would likely be too cold for a comfortable and safe swimming experience.

In [62]:
# Let's check another example
user_prompt = "I would like to explore my city tomorrow, i'm wondering if i need to carry an umbrella tomorrow"
new_prompt = f"{prompt}\n\n{user_prompt}"
print(new_prompt)

You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get

In [63]:
_ = agent1(new_prompt)

USER: You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your
final answer should be highly specific to the observations you have from running
the actions. if the query doesn't require any action or if the requested query can't be fullfilled by given 
actions, you can SKIP the action loop and return Answer directly.

Thought: Describe your thoughts about the question you have been asked.
Action: run one of the actions available to you - then return PAUSE.
PAUSE
Observation: will be the result of running those actions.

Available actions:
get_current_weather: E.g. get_current_weather: "Salt Lake City" Returns the current weather of the location 
specified.
get_location: E.g. get_location: "null" Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: get_location: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: get_current_weather: New York City
PAUSE

You'll then be called again with something like this:
Observation: { location: "New York City, NY", forecast: ["sunny"] }

You then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding 
areas.>

I would like to explore my city tomorrow, i'm wondering if i need to carry an umbrella tomorrow

AI: Thought: To determine if the user needs to carry an umbrella tomorrow, I need to know their location and the 
weather forecast for that location.
Action: get_location: null
PAUSE

In [64]:
# Since the AI is now requesting the tool for getting location, we can mimic the tool output back to the model
_ = agent1(get_location)

USER: Helsinki

AI: Thought: Now that I know the user is in Helsinki, I need to get the weather forecast for Helsinki to determine 
if they need to carry an umbrella tomorrow.
Action: get_current_weather: Helsinki
PAUSE

In [65]:
# Now the AI will request the tool for getting the current weather
_ = agent1(get_current_weather)

USER: { location: "Helsinki", forecast: ["Snowy"] }

AI: Thought: Since the forecast for Helsinki is snowy, it is likely that precipitation will occur. Therefore, it 
would be wise to carry an umbrella.
Answer: Yes, given the snowy forecast for Helsinki tomorrow, you should definitely carry an umbrella.

In [66]:
# now let's ask a question that doesn't require any action
user_prompt = "Can you tell me the time in Helsinki?"
_ = agent1(user_prompt)

USER: Can you tell me the time in Helsinki?

AI: Thought: I do not have the capability to tell you the current time in Helsinki. I can only get the current 
weather.
Answer: I am sorry, I cannot tell you the current time in Helsinki. I can only provide weather information.

In [67]:
# one more example
user_prompt = "My name is Venkat, how about you?"
_ = agent1(user_prompt)

USER: My name is Venkat, how about you?

AI: Thought: This question is conversational and doesn't require any action. I can respond directly.
Answer: As a large language model, I don't have a name in the same way humans do. You can call me whatever you 
like, or simply refer to me as the assistant.